# Experiment 007: Rotation Optimization + Backward Propagation

Build on exp_006's validated ensemble with:
1. Rotation optimization - rotate entire configuration to minimize bbox
2. Backward propagation from MULTIPLE starting points
3. Strict validation (1e18 scaling)

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
from scipy.optimize import minimize_scalar
from scipy.spatial import ConvexHull
from numba import njit
import os
import json
import time
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 30
SCALE_FACTOR = Decimal('1e18')

print("Setup complete")

Setup complete


In [2]:
# Tree shape vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def create_tree_polygon(x, y, angle):
    """Create tree polygon for scoring."""
    x, y, angle = float(x), float(y), float(angle)
    coords = list(zip(TX, TY))
    poly = Polygon(coords)
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

def create_high_precision_tree(x, y, angle):
    """Create tree polygon with 1e18 scaling for validation."""
    x = Decimal(str(x))
    y = Decimal(str(y))
    angle = Decimal(str(angle))
    
    sf = SCALE_FACTOR
    vertices = [
        (float(Decimal('0.0') * sf), float(Decimal('0.8') * sf)),
        (float(Decimal('0.125') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('0.0625') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('0.2') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('0.1') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('0.35') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('0.075') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('0.075') * sf), float(Decimal('-0.2') * sf)),
        (float(Decimal('-0.075') * sf), float(Decimal('-0.2') * sf)),
        (float(Decimal('-0.075') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('-0.35') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('-0.1') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('-0.2') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('-0.0625') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('-0.125') * sf), float(Decimal('0.5') * sf)),
    ]
    poly = Polygon(vertices)
    poly = affinity.rotate(poly, float(angle), origin=(0, 0))
    poly = affinity.translate(poly, xoff=float(x * sf), yoff=float(y * sf))
    return poly

def validate_no_overlap_strict(trees_data):
    """Check for overlaps using 1e18 scaling."""
    if len(trees_data) <= 1:
        return True, "OK"
    polygons = [create_high_precision_tree(t['x'], t['y'], t['deg']) for t in trees_data]
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                return False, f"Trees {i} and {j} overlap"
    return True, "OK"

def get_bbox_side(trees):
    """Get bounding box side length."""
    if len(trees) == 0:
        return 0
    polygons = [create_tree_polygon(t['x'], t['y'], t['deg']) for t in trees]
    union = unary_union(polygons)
    bounds = union.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def get_score(trees, n):
    """Get score contribution for N trees."""
    side = get_bbox_side(trees)
    return (side ** 2) / n

print("Core functions defined")

Core functions defined


In [3]:
# Fast scoring using numba
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group_fast(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * np.pi / 180.0
        c = np.cos(r); s = np.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    return np.array([float(str(v).replace("s","")) for v in a], np.float64)

tx, ty = make_polygon_template()
print("Fast scoring compiled")

Fast scoring compiled


In [4]:
# Load best ensemble from exp_006
print("Loading exp_006 validated ensemble...")
ensemble_path = '/home/code/experiments/006_validated_ensemble/submission.csv'
df = pd.read_csv(ensemble_path)
df["N"] = df["id"].astype(str).str.split("_").str[0].astype(int)

best_trees = {}
best_scores = {}

for n, g in df.groupby("N"):
    trees = []
    for _, row in g.iterrows():
        x = str(row['x']).replace('s', '')
        y = str(row['y']).replace('s', '')
        deg = str(row['deg']).replace('s', '')
        trees.append({'x': x, 'y': y, 'deg': deg})
    best_trees[n] = trees
    
    xs = strip(g["x"].to_numpy())
    ys = strip(g["y"].to_numpy())
    ds = strip(g["deg"].to_numpy())
    best_scores[n] = score_group_fast(xs, ys, ds, tx, ty)

baseline_total = sum(best_scores.values())
print(f"Loaded ensemble with score: {baseline_total:.6f}")

Loading exp_006 validated ensemble...


Loaded ensemble with score: 70.615744


In [5]:
# STEP 1: Rotation Optimization
# Rotate entire configuration to minimize bounding box
print("\n" + "=" * 60)
print("STEP 1: ROTATION OPTIMIZATION")
print("=" * 60)

def get_all_polygon_points(trees):
    """Get all vertices from all tree polygons."""
    all_points = []
    for t in trees:
        poly = create_tree_polygon(t['x'], t['y'], t['deg'])
        coords = list(poly.exterior.coords)[:-1]  # Exclude closing point
        all_points.extend(coords)
    return np.array(all_points)

def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bbox side after rotating all points by angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])
    rotated_points = points.dot(rot_matrix.T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def rotate_trees(trees, angle_deg):
    """Rotate all trees by angle around origin."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    
    rotated_trees = []
    for t in trees:
        x = float(t['x'])
        y = float(t['y'])
        deg = float(t['deg'])
        
        # Rotate position
        new_x = c * x - s * y
        new_y = s * x + c * y
        
        # Add rotation to angle
        new_deg = deg + angle_deg
        
        rotated_trees.append({
            'x': str(new_x),
            'y': str(new_y),
            'deg': str(new_deg)
        })
    
    return rotated_trees

def optimize_rotation_for_n(trees, n):
    """Find optimal rotation angle for a configuration."""
    if len(trees) <= 1:
        return trees, 0, False
    
    points = get_all_polygon_points(trees)
    current_side = calculate_bbox_side_at_angle(0, points)
    current_score = (current_side ** 2) / n
    
    # Search for optimal angle
    best_angle = 0
    best_side = current_side
    
    # Try angles from 0 to 90 (due to symmetry)
    for angle in np.linspace(0, 90, 181):  # 0.5 degree increments
        side = calculate_bbox_side_at_angle(angle, points)
        if side < best_side - 1e-10:
            best_side = side
            best_angle = angle
    
    if best_angle > 0.1:  # Only rotate if meaningful improvement
        rotated_trees = rotate_trees(trees, best_angle)
        new_score = (best_side ** 2) / n
        
        # Validate no overlaps
        is_valid, _ = validate_no_overlap_strict(rotated_trees)
        if is_valid and new_score < current_score - 1e-9:
            return rotated_trees, new_score, True
    
    return trees, current_score, False

print("Rotation optimization functions defined")


STEP 1: ROTATION OPTIMIZATION
Rotation optimization functions defined


In [6]:
# Apply rotation optimization to all N values
print("\nApplying rotation optimization to all N values...")

rotation_improvements = []
start_time = time.time()

for n in range(2, 201):  # Skip N=1 (already optimal)
    trees = best_trees[n]
    old_score = best_scores[n]
    
    new_trees, new_score, improved = optimize_rotation_for_n(trees, n)
    
    if improved:
        best_trees[n] = new_trees
        best_scores[n] = new_score
        improvement = old_score - new_score
        rotation_improvements.append((n, improvement))
        print(f"  N={n:3d}: improved by {improvement:.6f}")
    
    if n % 50 == 0:
        print(f"  Progress: N={n}/200")

print(f"\nRotation optimization complete in {time.time() - start_time:.1f}s")
print(f"N values improved: {len(rotation_improvements)}")
if rotation_improvements:
    total_rot_improvement = sum(imp for _, imp in rotation_improvements)
    print(f"Total improvement: {total_rot_improvement:.6f}")


Applying rotation optimization to all N values...


  Progress: N=50/200


  Progress: N=100/200


  Progress: N=150/200


  Progress: N=200/200

Rotation optimization complete in 4.1s
N values improved: 0


In [ ]:
# STEP 2: Backward Propagation from multiple sources
print("\n" + "=" * 60)
print("STEP 2: BACKWARD PROPAGATION")
print("=" * 60)

def get_bbox_touching_indices(trees):
    """Find indices of trees touching the bounding box boundary."""
    polygons = [create_tree_polygon(t['x'], t['y'], t['deg']) for t in trees]
    
    all_bounds = [p.bounds for p in polygons]
    minx = min(b[0] for b in all_bounds)
    miny = min(b[1] for b in all_bounds)
    maxx = max(b[2] for b in all_bounds)
    maxy = max(b[3] for b in all_bounds)
    
    touching = []
    for i, b in enumerate(all_bounds):
        if (abs(b[0] - minx) < 1e-6 or abs(b[1] - miny) < 1e-6 or
            abs(b[2] - maxx) < 1e-6 or abs(b[3] - maxy) < 1e-6):
            touching.append(i)
    
    return touching

def backward_propagation(trees_by_n, scores_by_n):
    """Try removing boundary trees from N to improve N-1."""
    improvements = []
    
    for n in range(200, 2, -1):
        trees = trees_by_n[n]
        target_n = n - 1
        current_best = scores_by_n[target_n]
        
        touching_indices = get_bbox_touching_indices(trees)
        
        for idx in touching_indices:
            # Create candidate by removing this tree
            candidate = [t for i, t in enumerate(trees) if i != idx]
            
            # Re-index
            for i, t in enumerate(candidate):
                t['idx'] = i
            
            # Calculate score
            candidate_score = get_score(candidate, target_n)
            
            if candidate_score < current_best - 1e-9:
                # Validate
                is_valid, _ = validate_no_overlap_strict(candidate)
                if is_valid:
                    trees_by_n[target_n] = candidate
                    scores_by_n[target_n] = candidate_score
                    improvement = current_best - candidate_score
                    improvements.append((target_n, improvement))
                    print(f"  N={target_n}: improved by {improvement:.6f}")
                    break
    
    return improvements

print("Running backward propagation...")
backprop_improvements = backward_propagation(best_trees, best_scores)
print(f"\nBackward propagation improved {len(backprop_improvements)} N values")

In [ ]:
# Calculate final score
final_total = sum(best_scores.values())
print(f"\n" + "=" * 60)
print("RESULTS")
print("=" * 60)
print(f"Original ensemble score: {baseline_total:.6f}")
print(f"After rotation optimization: {sum(best_scores.values()):.6f}")
print(f"Total improvement: {baseline_total - final_total:.6f}")

In [ ]:
# Final validation
print("\n" + "=" * 60)
print("FINAL VALIDATION")
print("=" * 60)

final_overlaps = []
for n in range(1, 201):
    is_valid, msg = validate_no_overlap_strict(best_trees[n])
    if not is_valid:
        final_overlaps.append((n, msg))

if final_overlaps:
    print(f"WARNING: {len(final_overlaps)} N values have overlaps!")
    for n, msg in final_overlaps[:10]:
        print(f"  N={n}: {msg}")
else:
    print("✅ All N values pass strict validation!")

In [ ]:
# Create submission
print("\n" + "=" * 60)
print("CREATE SUBMISSION")
print("=" * 60)

rows = []
for n in range(1, 201):
    trees = best_trees[n]
    for i, t in enumerate(trees):
        rows.append({
            'id': f"{n:03d}_{i}",
            'x': f"s{t['x']}",
            'y': f"s{t['y']}",
            'deg': f"s{t['deg']}"
        })

submission_df = pd.DataFrame(rows)
print(f"Submission shape: {submission_df.shape}")

submission_df.to_csv('/home/code/experiments/007_rotation_backprop/submission.csv', index=False)
submission_df.to_csv('/home/submission/submission.csv', index=False)
print("Submission saved!")

In [ ]:
# Save metrics
metrics = {
    'cv_score': final_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - final_total,
    'rotation_improvements': len(rotation_improvements),
    'backprop_improvements': len(backprop_improvements),
    'final_overlaps': len(final_overlaps),
    'target': 68.888293,
    'gap': final_total - 68.888293
}

with open('/home/code/experiments/007_rotation_backprop/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("\nMetrics saved!")
print(f"\n" + "=" * 60)
print("FINAL RESULTS")
print("=" * 60)
print(f"Original score: {baseline_total:.6f}")
print(f"Final score: {final_total:.6f}")
print(f"Improvement: {baseline_total - final_total:.6f}")
print(f"Target: 68.888293")
print(f"Gap to target: {final_total - 68.888293:.6f}")